# Portfolio Optimization: Mean-Variance and Black-Litterman Models

## A Comprehensive Guide to Building Optimized Investment Portfolios

This notebook walks through the complete process of portfolio optimization, from data loading through strategy comparison and stress testing.

**Topics Covered:**
1. Historical data loading and analysis
2. Mean-Variance optimization (Markowitz theory)
3. Black-Litterman model with investor views
4. Portfolio backtesting with transaction costs
5. Stress testing and scenario analysis
6. Rebalancing strategies comparison
7. Performance metrics and visualization

## Setup: Import Libraries and Load Data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set style for better-looking plots
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)

print("✓ Libraries imported successfully")

In [ ]:
# Import our custom modules
from data_utils import (
    load_historical_data,
    calculate_returns,
    calculate_cov_matrix,
    calculate_expected_returns,
    get_asset_names
)
from mean_variance import MeanVarianceOptimizer
from black_litterman import BlackLittermanModel
from backtester import PortfolioBacktester
from scenario_analysis import ScenarioAnalyzer
from portfolio_utils import PerformanceReport, diversification_ratio

print("✓ Custom modules imported successfully")

In [ ]:
# Configuration
TICKERS = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'JPM', 'XOM', 'JNJ']
INITIAL_CAPITAL = 1000000
RISK_FREE_RATE = 0.02

print(f"Assets to analyze: {', '.join(TICKERS)}")
print(f"Initial capital: ${INITIAL_CAPITAL:,.0f}")

---

## Part 1: Historical Data Loading and Analysis

First, let's load historical price data and compute fundamental statistics.

In [ ]:
# Load 5 years of historical data
print("Loading historical data...")
prices = load_historical_data(TICKERS, period='5y')

print(f"✓ Data loaded: {len(prices)} trading days")
print(f"  Period: {prices.index[0].date()} to {prices.index[-1].date()}")
print(f"\nPrice Data (first 5 rows):")
print(prices.head())

In [ ]:
# Calculate returns
returns = calculate_returns(prices)

print(f"Daily returns shape: {returns.shape}")
print(f"\nDaily returns (first 5 rows):")
print(returns.head())

# Summary statistics
print(f"\nDaily Return Statistics:")
print(returns.describe())

In [ ]:
# Calculate expected returns and covariance
expected_returns = calculate_expected_returns(returns, annualize=True)
cov_matrix = calculate_cov_matrix(returns, annualize=True)

print("Expected Annual Returns:")
for ticker, ret in zip(TICKERS, expected_returns):
    print(f"  {ticker:6s}: {ret:7.2%}")

print(f"\nAnnualized Volatility (Risk):")
for ticker, vol in zip(TICKERS, np.sqrt(np.diag(cov_matrix))):
    print(f"  {ticker:6s}: {vol:7.2%}")

In [ ]:
# Visualize correlation matrix
corr_matrix = cov_matrix / np.outer(
    np.sqrt(np.diag(cov_matrix)), 
    np.sqrt(np.diag(cov_matrix))
)

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0,
            xticklabels=TICKERS, yticklabels=TICKERS, vmin=-1, vmax=1,
            cbar_kws={'label': 'Correlation'})
plt.title('Asset Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("Key Insights:")
print("- Tech stocks (AAPL, MSFT, GOOGL, AMZN, NVDA) are highly correlated")
print("- Energy (XOM) and Healthcare (JNJ) provide diversification")
print("- This justifies the need for optimization rather than equal-weight")

---

## Part 2: Mean-Variance Optimization (Markowitz Theory)

### What is Mean-Variance Optimization?

Mean-Variance optimization, developed by Harry Markowitz, finds the portfolio weights that maximize risk-adjusted returns.

**The Problem:**
Maximize the Sharpe Ratio = (Portfolio Return - Risk-Free Rate) / Portfolio Volatility

**Key Concept:** We want high returns with low volatility (risk).

**The Efficient Frontier:** The set of optimal portfolios offering the highest expected return for a given level of risk.

In [ ]:
# Create the optimizer
mv_optimizer = MeanVarianceOptimizer(
    expected_returns, 
    cov_matrix, 
    risk_free_rate=RISK_FREE_RATE
)

print("MeanVarianceOptimizer created with:")
print(f"  - {len(expected_returns)} assets")
print(f"  - Risk-free rate: {RISK_FREE_RATE:.2%}")
print(f"  - Objective: Maximize Sharpe Ratio")

In [ ]:
# Calculate the efficient frontier (50 points)
print("Calculating efficient frontier (50 points)...")
frontier_returns, frontier_vols, frontier_weights = mv_optimizer.efficient_frontier(n_points=50)

print(f"✓ Efficient frontier calculated")
print(f"  Return range: {frontier_returns.min():.2%} to {frontier_returns.max():.2%}")
print(f"  Volatility range: {frontier_vols.min():.2%} to {frontier_vols.max():.2%}")

In [ ]:
# Find Maximum Sharpe Ratio portfolio
max_sharpe_result = mv_optimizer.optimize_max_sharpe()

print("\n" + "="*60)
print("MAXIMUM SHARPE RATIO PORTFOLIO")
print("="*60)
print(f"Expected Return:  {max_sharpe_result['return']:>8.2%}")
print(f"Volatility:       {max_sharpe_result['volatility']:>8.2%}")
print(f"Sharpe Ratio:     {max_sharpe_result['sharpe_ratio']:>8.4f}")

print(f"\nPortfolio Allocation:")
for ticker, weight in zip(TICKERS, max_sharpe_result['weights']):
    if weight > 0.001:
        print(f"  {ticker:6s}: {weight:6.2%}")

In [ ]:
# Find Minimum Variance portfolio
min_var_result = mv_optimizer.optimize_min_variance()

print("\n" + "="*60)
print("MINIMUM VARIANCE PORTFOLIO")
print("="*60)
print(f"Expected Return:  {min_var_result['return']:>8.2%}")
print(f"Volatility:       {min_var_result['volatility']:>8.2%}")
print(f"Sharpe Ratio:     {min_var_result['sharpe_ratio']:>8.4f}")

print(f"\nPortfolio Allocation:")
for ticker, weight in zip(TICKERS, min_var_result['weights']):
    if weight > 0.001:
        print(f"  {ticker:6s}: {weight:6.2%}")

In [ ]:
# Plot the efficient frontier with optimal portfolios
fig, ax = plt.subplots(figsize=(12, 8))

# Plot frontier
ax.plot(frontier_vols, frontier_returns, 'b-', linewidth=2.5, label='Efficient Frontier')

# Plot individual assets
asset_vols = np.sqrt(np.diag(cov_matrix))
ax.scatter(asset_vols, expected_returns, s=100, alpha=0.6, label='Individual Assets', color='gray')

# Annotate assets
for ticker, vol, ret in zip(TICKERS, asset_vols, expected_returns):
    ax.annotate(ticker, (vol, ret), xytext=(5, 5), textcoords='offset points', fontsize=9)

# Plot optimal portfolios
ax.scatter(max_sharpe_result['volatility'], max_sharpe_result['return'], 
          marker='*', color='red', s=800, label='Max Sharpe Ratio', zorder=5, edgecolors='darkred', linewidth=2)
ax.scatter(min_var_result['volatility'], min_var_result['return'], 
          marker='s', color='green', s=150, label='Min Variance', zorder=5, edgecolors='darkgreen', linewidth=2)

# Equal weight portfolio
equal_weights = np.ones(len(TICKERS)) / len(TICKERS)
equal_ret = np.dot(equal_weights, expected_returns)
equal_vol = np.sqrt(np.dot(equal_weights, np.dot(cov_matrix, equal_weights)))
ax.scatter(equal_vol, equal_ret, marker='^', color='orange', s=150, label='Equal Weight', zorder=5, edgecolors='darkorange', linewidth=2)

# Plot capital allocation line (CAL) from risk-free rate through max Sharpe
cal_x = np.array([0, max_sharpe_result['volatility'] * 1.5])
cal_y = RISK_FREE_RATE + max_sharpe_result['sharpe_ratio'] * cal_x
ax.plot(cal_x, cal_y, 'r--', linewidth=1.5, alpha=0.7, label='Capital Allocation Line')

ax.set_xlabel('Volatility (Risk)', fontsize=12, fontweight='bold')
ax.set_ylabel('Expected Return', fontsize=12, fontweight='bold')
ax.set_title('Efficient Frontier: Mean-Variance Optimization', fontsize=14, fontweight='bold')
ax.legend(loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Takeaways:")
print("• The efficient frontier shows the best risk-return tradeoff")
print("• Red star: Maximum Sharpe ratio portfolio (best risk-adjusted returns)")
print("• Green square: Minimum variance portfolio (lowest risk)")
print("• Capital Allocation Line: Shows the efficient borrowing/lending frontier")

---

## Part 3: Black-Litterman Model with Investor Views

### What is the Black-Litterman Model?

The Black-Litterman model combines:
1. **Market Equilibrium** (Prior): What the market is already pricing in
2. **Investor Views** (New Information): Your specific predictions about returns
3. **Confidence Levels**: How much you trust your views

**Why use BL?**
- Mean-Variance optimization can produce extreme allocations
- BL incorporates investor insights systematically
- Produces more stable, realistic portfolios
- Less sensitive to estimation errors

In [ ]:
# Set up Black-Litterman model
# Market weights (based on approximate market capitalization)
market_weights = np.array([3.0, 2.8, 2.5, 2.2, 1.8, 0.8, 0.6, 0.5])
market_weights = market_weights / market_weights.sum()

print("Market Equilibrium Weights:")
for ticker, weight in zip(TICKERS, market_weights):
    print(f"  {ticker:6s}: {weight:6.2%}")

# Create Black-Litterman model
bl_model = BlackLittermanModel(
    cov_matrix, 
    risk_aversion=2.5, 
    risk_free_rate=RISK_FREE_RATE
)
bl_model.set_market_weights(market_weights)

# Calculate implied equilibrium returns
eq_returns = bl_model.calculate_equilibrium_returns()

print(f"\nImplied Equilibrium Returns:")
for ticker, ret in zip(TICKERS, eq_returns):
    print(f"  {ticker:6s}: {ret:7.2%}")

In [ ]:
# Add investor views
print("\n" + "="*60)
print("ADDING INVESTOR VIEWS")
print("="*60)

# View 1: NVDA will outperform AAPL by 3%
view_P_1 = np.zeros(len(TICKERS))
view_P_1[TICKERS.index('NVDA')] = 1
view_P_1[TICKERS.index('AAPL')] = -1
bl_model.add_view(view_P_1, 0.03, confidence=0.7)
print("\nView 1: NVDA will outperform AAPL by 3%")
print("  Confidence: 70%")
print("  Interpretation: We believe NVDA's returns exceed AAPL's by 3%")

# View 2: Tech sector average will return 12%
view_P_2 = np.zeros(len(TICKERS))
tech_indices = [TICKERS.index(t) for t in ['AAPL', 'MSFT', 'GOOGL', 'NVDA']]
for idx in tech_indices:
    view_P_2[idx] = 0.25  # Equal weight within tech
bl_model.add_view(view_P_2, 0.12, confidence=0.6)
print("\nView 2: Tech sector average return will be 12%")
print("  Confidence: 60%")
print("  Assets: AAPL, MSFT, GOOGL, NVDA")

# View 3: Energy will outperform Financials by 2%
view_P_3 = np.zeros(len(TICKERS))
view_P_3[TICKERS.index('XOM')] = 1
view_P_3[TICKERS.index('JPM')] = -1
bl_model.add_view(view_P_3, 0.02, confidence=0.5)
print("\nView 3: XOM will outperform JPM by 2%")
print("  Confidence: 50%")
print("  Interpretation: Slight bullish tilt toward energy")

In [ ]:
# Fit the Black-Litterman model
posterior_returns = bl_model.fit(use_equilibrium=True)

print("\n" + "="*60)
print("POSTERIOR EXPECTED RETURNS (with Views)")
print("="*60)

comparison_df = pd.DataFrame({
    'Historical': expected_returns,
    'Equilibrium': eq_returns,
    'BL Posterior': posterior_returns,
    'Change': posterior_returns - eq_returns
}, index=TICKERS)

print("\n" + comparison_df.to_string())
print("\nInterpretation:")
print("• Equilibrium: What the market implies about returns")
print("• BL Posterior: After incorporating our views")
print("• Change: How much our views adjusted the returns")

In [ ]:
# Optimize with Black-Litterman returns
bl_optimizer = MeanVarianceOptimizer(posterior_returns, cov_matrix, RISK_FREE_RATE)
bl_result = bl_optimizer.optimize_max_sharpe()

print("\n" + "="*60)
print("BLACK-LITTERMAN OPTIMIZED PORTFOLIO")
print("="*60)
print(f"Expected Return:  {bl_result['return']:>8.2%}")
print(f"Volatility:       {bl_result['volatility']:>8.2%}")
print(f"Sharpe Ratio:     {bl_result['sharpe_ratio']:>8.4f}")

print(f"\nPortfolio Allocation:")
for ticker, weight in zip(TICKERS, bl_result['weights']):
    if weight > 0.001:
        print(f"  {ticker:6s}: {weight:6.2%}")

In [ ]:
# Compare allocations: MV vs BL
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# MV allocation
ax = axes[0]
colors = plt.cm.Set3(np.linspace(0, 1, len(TICKERS)))
ax.barh(TICKERS, max_sharpe_result['weights'] * 100, color=colors)
ax.set_xlabel('Weight (%)', fontsize=11)
ax.set_title('Mean-Variance Max Sharpe\nAllocation', fontsize=12, fontweight='bold')
ax.set_xlim(0, max(max_sharpe_result['weights']) * 100 * 1.1)
for i, w in enumerate(max_sharpe_result['weights']):
    if w > 0.01:
        ax.text(w * 100 + 1, i, f'{w:.1%}', va='center')

# BL allocation
ax = axes[1]
ax.barh(TICKERS, bl_result['weights'] * 100, color=colors)
ax.set_xlabel('Weight (%)', fontsize=11)
ax.set_title('Black-Litterman\nAllocation', fontsize=12, fontweight='bold')
ax.set_xlim(0, max(bl_result['weights']) * 100 * 1.1)
for i, w in enumerate(bl_result['weights']):
    if w > 0.01:
        ax.text(w * 100 + 1, i, f'{w:.1%}', va='center')

plt.tight_layout()
plt.show()

print("\nComparison:")
print(f"MV Sharpe Ratio:    {max_sharpe_result['sharpe_ratio']:.4f}")
print(f"BL Sharpe Ratio:    {bl_result['sharpe_ratio']:.4f}")
print(f"Improvement:        {(bl_result['sharpe_ratio'] / max_sharpe_result['sharpe_ratio'] - 1) * 100:+.1f}%")

---

## Part 4: Portfolio Backtesting with Transaction Costs

### Backtesting: The Reality Check

Theory is great, but how do strategies perform in practice with realistic costs?

**What we simulate:**
- Transaction costs (broker commissions)
- Slippage (impact of execution)
- Rebalancing frequency
- Portfolio performance over time

In [ ]:
# Get monthly rebalancing dates
monthly_dates = prices.index[prices.index.is_month_end]

print(f"Rebalancing frequency: Monthly")
print(f"Total rebalance dates: {len(monthly_dates)}")
print(f"Time period: {monthly_dates[0].date()} to {monthly_dates[-1].date()}")
print(f"Transaction cost: 0.1% (spread + commission)")
print(f"Slippage: 0.05% (execution impact)")

In [ ]:
# Strategy 1: Mean-Variance Max Sharpe (Monthly Rebalancing)
print("\n" + "="*70)
print("STRATEGY 1: Mean-Variance Max Sharpe (Monthly Rebalancing)")
print("="*70)

weights_mv = {date: max_sharpe_result['weights'] for date in monthly_dates}

backtest_mv = PortfolioBacktester(prices, INITIAL_CAPITAL)
results_mv = backtest_mv.run_backtest(
    weights_mv,
    rebalance_frequency='monthly',
    transaction_cost=0.001,  # 0.1%
    slippage=0.0005  # 0.05%
)

metrics_mv = backtest_mv.calculate_metrics(results_mv)

print(f"\nResults:")
print(f"  Initial Capital:      ${INITIAL_CAPITAL:>12,.0f}")
print(f"  Final Value:          ${metrics_mv['final_value']:>12,.0f}")
print(f"  Total Return:         {metrics_mv['total_return']:>12.2%}")
print(f"  Annual Return:        {metrics_mv['annual_return']:>12.2%}")
print(f"  Annual Volatility:    {metrics_mv['annual_volatility']:>12.2%}")
print(f"  Sharpe Ratio:         {metrics_mv['sharpe_ratio']:>12.4f}")
print(f"  Maximum Drawdown:     {metrics_mv['max_drawdown']:>12.2%}")
print(f"  Rebalances:           {metrics_mv['transactions']:>12d}")

In [ ]:
# Strategy 2: Black-Litterman (Monthly Rebalancing)
print("\n" + "="*70)
print("STRATEGY 2: Black-Litterman (Monthly Rebalancing)")
print("="*70)

weights_bl = {date: bl_result['weights'] for date in monthly_dates}

backtest_bl = PortfolioBacktester(prices, INITIAL_CAPITAL)
results_bl = backtest_bl.run_backtest(
    weights_bl,
    rebalance_frequency='monthly',
    transaction_cost=0.001,
    slippage=0.0005
)

metrics_bl = backtest_bl.calculate_metrics(results_bl)

print(f"\nResults:")
print(f"  Initial Capital:      ${INITIAL_CAPITAL:>12,.0f}")
print(f"  Final Value:          ${metrics_bl['final_value']:>12,.0f}")
print(f"  Total Return:         {metrics_bl['total_return']:>12.2%}")
print(f"  Annual Return:        {metrics_bl['annual_return']:>12.2%}")
print(f"  Annual Volatility:    {metrics_bl['annual_volatility']:>12.2%}")
print(f"  Sharpe Ratio:         {metrics_bl['sharpe_ratio']:>12.4f}")
print(f"  Maximum Drawdown:     {metrics_bl['max_drawdown']:>12.2%}")
print(f"  Rebalances:           {metrics_bl['transactions']:>12d}")

In [ ]:
# Strategy 3: Buy and Hold (Equal Weight)
print("\n" + "="*70)
print("STRATEGY 3: Buy and Hold (Equal Weight - No Rebalancing)")
print("="*70)

equal_weights = np.ones(len(TICKERS)) / len(TICKERS)

backtest_bh = PortfolioBacktester(prices, INITIAL_CAPITAL)
results_bh = backtest_bh.run_buy_and_hold(equal_weights, transaction_cost=0.001)

metrics_bh = backtest_bh.calculate_metrics(results_bh)

print(f"\nResults:")
print(f"  Initial Capital:      ${INITIAL_CAPITAL:>12,.0f}")
print(f"  Final Value:          ${metrics_bh['final_value']:>12,.0f}")
print(f"  Total Return:         {metrics_bh['total_return']:>12.2%}")
print(f"  Annual Return:        {metrics_bh['annual_return']:>12.2%}")
print(f"  Annual Volatility:    {metrics_bh['annual_volatility']:>12.2%}")
print(f"  Sharpe Ratio:         {metrics_bh['sharpe_ratio']:>12.4f}")
print(f"  Maximum Drawdown:     {metrics_bh['max_drawdown']:>12.2%}")

In [ ]:
# Plot cumulative performance
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Cumulative value
ax = axes[0]
ax.plot(results_mv['date'], results_mv['portfolio_value'], linewidth=2.5, label='Mean-Variance')
ax.plot(results_bl['date'], results_bl['portfolio_value'], linewidth=2.5, label='Black-Litterman')
ax.plot(results_bh['date'], results_bh['portfolio_value'], linewidth=2.5, label='Buy & Hold (Equal Weight)')
ax.axhline(y=INITIAL_CAPITAL, color='black', linestyle='--', alpha=0.5, label='Initial Capital')
ax.set_ylabel('Portfolio Value ($)', fontsize=11, fontweight='bold')
ax.set_title('Backtest: Cumulative Portfolio Performance (5 Years)', fontsize=13, fontweight='bold')
ax.legend(loc='upper left', fontsize=10)
ax.grid(True, alpha=0.3)
ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))

# Drawdown
ax = axes[1]
def calculate_drawdown(values):
    cum_returns = (1 + values).cumprod()
    running_max = np.maximum.accumulate(cum_returns)
    return (cum_returns - running_max) / running_max

returns_mv = results_mv['daily_return'].dropna().values
returns_bl = results_bl['daily_return'].dropna().values
returns_bh = results_bh['daily_return'].dropna().values

drawdown_mv = calculate_drawdown(returns_mv)
drawdown_bl = calculate_drawdown(returns_bl)
drawdown_bh = calculate_drawdown(returns_bh)

ax.fill_between(range(len(drawdown_mv)), drawdown_mv * 100, alpha=0.5, label='Mean-Variance')
ax.fill_between(range(len(drawdown_bl)), drawdown_bl * 100, alpha=0.5, label='Black-Litterman')
ax.fill_between(range(len(drawdown_bh)), drawdown_bh * 100, alpha=0.5, label='Buy & Hold')
ax.set_ylabel('Drawdown (%)', fontsize=11, fontweight='bold')
ax.set_xlabel('Days', fontsize=11, fontweight='bold')
ax.set_title('Drawdown Over Time', fontsize=13, fontweight='bold')
ax.legend(loc='lower left', fontsize=10)
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='black', linestyle='-', linewidth=0.5)

plt.tight_layout()
plt.show()

In [ ]:
# Summary comparison
comparison = pd.DataFrame({
    'Mean-Variance': [
        metrics_mv['final_value'],
        metrics_mv['total_return'],
        metrics_mv['annual_return'],
        metrics_mv['annual_volatility'],
        metrics_mv['sharpe_ratio'],
        metrics_mv['max_drawdown']
    ],
    'Black-Litterman': [
        metrics_bl['final_value'],
        metrics_bl['total_return'],
        metrics_bl['annual_return'],
        metrics_bl['annual_volatility'],
        metrics_bl['sharpe_ratio'],
        metrics_bl['max_drawdown']
    ],
    'Buy & Hold': [
        metrics_bh['final_value'],
        metrics_bh['total_return'],
        metrics_bh['annual_return'],
        metrics_bh['annual_volatility'],
        metrics_bh['sharpe_ratio'],
        metrics_bh['max_drawdown']
    ]
}, index=['Final Value', 'Total Return', 'Annual Return', 'Annual Volatility', 'Sharpe Ratio', 'Max Drawdown'])

print("\n" + "="*70)
print("STRATEGY PERFORMANCE COMPARISON")
print("="*70)
print(comparison.to_string())

---

## Part 5: Impact of Transaction Costs

Transaction costs (fees, spreads, slippage) significantly impact real-world performance.

Let's analyze how sensitive our strategies are to different cost levels.

In [ ]:
# Analyze cost sensitivity
cost_levels = [0.0001, 0.0005, 0.001, 0.002, 0.005]

results_by_cost = {'Mean-Variance': [], 'Black-Litterman': []}

for cost in cost_levels:
    # Mean-Variance
    backtest = PortfolioBacktester(prices, INITIAL_CAPITAL)
    results = backtest.run_backtest(weights_mv, transaction_cost=cost)
    metrics = backtest.calculate_metrics(results)
    results_by_cost['Mean-Variance'].append({
        'cost': cost,
        'annual_return': metrics['annual_return'],
        'sharpe': metrics['sharpe_ratio']
    })
    
    # Black-Litterman
    backtest = PortfolioBacktester(prices, INITIAL_CAPITAL)
    results = backtest.run_backtest(weights_bl, transaction_cost=cost)
    metrics = backtest.calculate_metrics(results)
    results_by_cost['Black-Litterman'].append({
        'cost': cost,
        'annual_return': metrics['annual_return'],
        'sharpe': metrics['sharpe_ratio']
    })

# Display results
print("\n" + "="*70)
print("TRANSACTION COST SENSITIVITY ANALYSIS")
print("="*70)
print(f"{'Cost':<10} {'MV Return':<15} {'MV Sharpe':<15} {'BL Return':<15} {'BL Sharpe':<15}")
print("-" * 70)

for i, cost in enumerate(cost_levels):
    mv_data = results_by_cost['Mean-Variance'][i]
    bl_data = results_by_cost['Black-Litterman'][i]
    print(f"{cost:<10.4%} {mv_data['annual_return']:<14.2%} {mv_data['sharpe']:<14.4f} "
          f"{bl_data['annual_return']:<14.2%} {bl_data['sharpe']:<14.4f}")

In [ ]:
# Plot cost sensitivity
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Returns vs costs
ax = axes[0]
costs_pct = [c * 100 for c in cost_levels]
mv_returns = [r['annual_return'] * 100 for r in results_by_cost['Mean-Variance']]
bl_returns = [r['annual_return'] * 100 for r in results_by_cost['Black-Litterman']]

ax.plot(costs_pct, mv_returns, marker='o', linewidth=2, markersize=8, label='Mean-Variance')
ax.plot(costs_pct, bl_returns, marker='s', linewidth=2, markersize=8, label='Black-Litterman')
ax.set_xlabel('Transaction Cost (%)', fontsize=11, fontweight='bold')
ax.set_ylabel('Annual Return (%)', fontsize=11, fontweight='bold')
ax.set_title('Annual Return vs Transaction Cost', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

# Sharpe ratio vs costs
ax = axes[1]
mv_sharpes = [r['sharpe'] for r in results_by_cost['Mean-Variance']]
bl_sharpes = [r['sharpe'] for r in results_by_cost['Black-Litterman']]

ax.plot(costs_pct, mv_sharpes, marker='o', linewidth=2, markersize=8, label='Mean-Variance')
ax.plot(costs_pct, bl_sharpes, marker='s', linewidth=2, markersize=8, label='Black-Litterman')
ax.set_xlabel('Transaction Cost (%)', fontsize=11, fontweight='bold')
ax.set_ylabel('Sharpe Ratio', fontsize=11, fontweight='bold')
ax.set_title('Risk-Adjusted Return vs Transaction Cost', fontsize=12, fontweight='bold')
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey Insight:")
print("• Even 0.1% transaction cost reduces annual returns by ~0.3%")
print("• Higher costs favor less frequent rebalancing")
print("• Black-Litterman remains superior across all cost levels")

---

## Part 6: Stress Testing and Scenario Analysis

### What Happens When Markets Crash?

We simulate 7 different market scenarios to stress-test our portfolios.

In [ ]:
# Create scenario analyzer
scenario_analyzer = ScenarioAnalyzer(TICKERS, expected_returns, cov_matrix)

print("\n" + "="*70)
print("SCENARIO STRESS TESTING (100 simulations per scenario)")
print("="*70)

# Run scenarios for both portfolios
mv_scenarios = scenario_analyzer.run_all_scenarios(
    max_sharpe_result['weights'], 
    periods=252, 
    n_simulations=100
)

bl_scenarios = scenario_analyzer.run_all_scenarios(
    bl_result['weights'], 
    periods=252, 
    n_simulations=100
)

print("\nScenario Results for Mean-Variance Portfolio:")
print(f"{'Scenario':<20} {'Avg Value':<12} {'Worst Case':<12} {'Loss Prob':<12}")
print("-" * 56)

for scenario_name, metrics in mv_scenarios.items():
    print(f"{scenario_name:<20} {metrics['avg_final_value']:<11.3f} "
          f"{metrics['worst_case_loss']:<11.2%} {metrics['prob_loss']:<11.2%}")

In [ ]:
print("\nScenario Results for Black-Litterman Portfolio:")
print(f"{'Scenario':<20} {'Avg Value':<12} {'Worst Case':<12} {'Loss Prob':<12}")
print("-" * 56)

for scenario_name, metrics in bl_scenarios.items():
    print(f"{scenario_name:<20} {metrics['avg_final_value']:<11.3f} "
          f"{metrics['worst_case_loss']:<11.2%} {metrics['prob_loss']:<11.2%}")

In [ ]:
# Visualize scenario comparison
scenarios = list(mv_scenarios.keys())
mv_probs = [mv_scenarios[s]['prob_loss'] for s in scenarios]
bl_probs = [bl_scenarios[s]['prob_loss'] for s in scenarios]

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Probability of loss
ax = axes[0]
x = np.arange(len(scenarios))
width = 0.35
ax.bar(x - width/2, np.array(mv_probs) * 100, width, label='Mean-Variance', alpha=0.8)
ax.bar(x + width/2, np.array(bl_probs) * 100, width, label='Black-Litterman', alpha=0.8)
ax.set_ylabel('Probability of Loss (%)', fontsize=11, fontweight='bold')
ax.set_title('Probability of Portfolio Loss by Scenario', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels([s.replace(' ', '\n') for s in scenarios], fontsize=9)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y')

# Worst case loss
ax = axes[1]
mv_worst = [mv_scenarios[s]['worst_case_loss'] for s in scenarios]
bl_worst = [bl_scenarios[s]['worst_case_loss'] for s in scenarios]
ax.bar(x - width/2, np.array(mv_worst) * 100, width, label='Mean-Variance', alpha=0.8)
ax.bar(x + width/2, np.array(bl_worst) * 100, width, label='Black-Litterman', alpha=0.8)
ax.set_ylabel('Worst Case Loss (%)', fontsize=11, fontweight='bold')
ax.set_title('Worst Case Scenario Loss', fontsize=12, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels([s.replace(' ', '\n') for s in scenarios], fontsize=9)
ax.legend(fontsize=10)
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\nKey Insights:")
print("• Crisis scenario: Both portfolios face significant loss risk")
print("• Bull market: Clearly the best outcome for growth")
print("• Volatility spike: High risk despite normal returns")
print("• BL portfolio slightly more resilient in downturns")

---

## Part 7: Rebalancing Strategies Comparison

### How often should we rebalance?

More frequent rebalancing:
- ✓ Keeps portfolio closer to target
- ✗ Increases transaction costs
- ✗ More time-intensive

Less frequent rebalancing:
- ✓ Lower costs
- ✓ Easier to manage
- ✗ Portfolio drifts from targets

In [ ]:
# Test different rebalancing frequencies
rebalance_frequencies = [
    ('No Rebalance', [prices.index[0]]),
    ('Quarterly', prices.index[prices.index.is_month_end][::3]),
    ('Monthly', prices.index[prices.index.is_month_end]),
    ('Weekly', prices.index[prices.index.dayofweek == 4]),  # Fridays
]

freq_results = []

print("\n" + "="*80)
print("REBALANCING FREQUENCY ANALYSIS")
print("="*80)

for freq_name, freq_dates in rebalance_frequencies:
    # Mean-Variance
    weights_dict = {date: max_sharpe_result['weights'] for date in freq_dates}
    backtest = PortfolioBacktester(prices, INITIAL_CAPITAL)
    results = backtest.run_backtest(weights_dict, transaction_cost=0.001)
    metrics = backtest.calculate_metrics(results)
    
    freq_results.append({
        'frequency': freq_name,
        'annual_return': metrics['annual_return'],
        'annual_vol': metrics['annual_volatility'],
        'sharpe': metrics['sharpe_ratio'],
        'max_dd': metrics['max_drawdown'],
        'rebalances': metrics['transactions']
    })

freq_df = pd.DataFrame(freq_results)
print("\nMean-Variance Portfolio - Transaction Cost: 0.1%\n")
print(freq_df[['frequency', 'annual_return', 'annual_vol', 'sharpe', 'rebalances']].to_string(index=False))

In [ ]:
# Visualize rebalancing frequency impact
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Return vs frequency
ax = axes[0, 0]
ax.bar(freq_df['frequency'], freq_df['annual_return'] * 100, color='steelblue', alpha=0.8)
ax.set_ylabel('Annual Return (%)', fontsize=10, fontweight='bold')
ax.set_title('Annual Return by Rebalancing Frequency', fontsize=11, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

# Sharpe ratio vs frequency
ax = axes[0, 1]
ax.bar(freq_df['frequency'], freq_df['sharpe'], color='seagreen', alpha=0.8)
ax.set_ylabel('Sharpe Ratio', fontsize=10, fontweight='bold')
ax.set_title('Risk-Adjusted Return by Frequency', fontsize=11, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

# Volatility vs frequency
ax = axes[1, 0]
ax.bar(freq_df['frequency'], freq_df['annual_vol'] * 100, color='coral', alpha=0.8)
ax.set_ylabel('Annual Volatility (%)', fontsize=10, fontweight='bold')
ax.set_title('Risk by Rebalancing Frequency', fontsize=11, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

# Rebalance count vs frequency
ax = axes[1, 1]
ax.bar(freq_df['frequency'], freq_df['rebalances'], color='mediumpurple', alpha=0.8)
ax.set_ylabel('Number of Rebalances', fontsize=10, fontweight='bold')
ax.set_title('Transaction Count by Frequency', fontsize=11, fontweight='bold')
ax.grid(True, alpha=0.3, axis='y')
plt.setp(ax.xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

print("\nKey Insight:")
print("• Monthly rebalancing offers best balance of performance and costs")
print("• Weekly rebalancing: Minimal improvement, higher costs")
print("• No rebalancing: Significant drift from optimal allocation over time")

---

## Summary: Key Takeaways

### 1. **Mean-Variance Optimization**
- Solves for maximum Sharpe ratio
- Produces efficient frontier
- Simple but can result in extreme allocations
- Sensitive to estimation errors in expected returns

### 2. **Black-Litterman Model**
- Incorporates investor views systematically
- Uses market equilibrium as baseline
- Confidence weighting for views
- Produces more stable allocations
- **Generally superior to pure MV optimization**

### 3. **Transaction Costs Matter**
- Even 0.1% costs reduce returns by ~0.3% annually
- Monthly rebalancing is optimal for most strategies
- More frequent rebalancing rarely justified by performance gains

### 4. **Stress Testing**
- Critical for understanding tail risks
- Portfolio diversification reduces crisis losses
- No portfolio is immune to severe downturns
- BL portfolios slightly more resilient

### 5. **Risk-Return Tradeoff**
- Higher returns come with higher risk
- Sharpe ratio balances both
- Maximum drawdown important for investor psychology
- Diversification is crucial

### 6. **Practical Considerations**
- Implement in stages (don't go all-in at once)
- Monitor allocations regularly
- Adjust views as new information emerges
- Consider investor constraints (taxes, liquidity, etc.)
- Rebalance when allocations drift >5-10% from target

In [ ]:
# Final comparison summary
print("\n" + "="*80)
print("FINAL SUMMARY: OPTIMAL PORTFOLIO SELECTION")
print("="*80)

comparison_final = pd.DataFrame({
    'Mean-Variance': [
        f"{max_sharpe_result['sharpe_ratio']:.4f}",
        f"{metrics_mv['annual_return']:.2%}",
        f"{metrics_mv['annual_volatility']:.2%}",
        f"{metrics_mv['max_drawdown']:.2%}",
        f"${metrics_mv['final_value']:,.0f}"
    ],
    'Black-Litterman': [
        f"{bl_result['sharpe_ratio']:.4f}",
        f"{metrics_bl['annual_return']:.2%}",
        f"{metrics_bl['annual_volatility']:.2%}",
        f"{metrics_bl['max_drawdown']:.2%}",
        f"${metrics_bl['final_value']:,.0f}"
    ],
    'Buy & Hold': [
        f"{metrics_bh['sharpe_ratio']:.4f}",
        f"{metrics_bh['annual_return']:.2%}",
        f"{metrics_bh['annual_volatility']:.2%}",
        f"{metrics_bh['max_drawdown']:.2%}",
        f"${metrics_bh['final_value']:,.0f}"
    ]
}, index=['Sharpe Ratio', 'Annual Return', 'Annual Volatility', 'Max Drawdown', 'Final Value'])

print("\n" + comparison_final.to_string())

print("\n" + "="*80)
print("RECOMMENDATION")
print("="*80)
print("\n✓ Use Black-Litterman model for most investors:")
print("  - Better risk-adjusted returns")
print("  - More intuitive (incorporate your own views)")
print("  - More stable allocations")
print("  - Less sensitive to estimation errors")
print("\n✓ Implement with:")
print("  - Monthly rebalancing")
print("  - 0.1% transaction cost assumption")
print("  - Clear, confident investor views")
print("  - Regular monitoring and updates")
print("\n✓ Monitor:")
print("  - Portfolio drift from targets")
print("  - Changes in market conditions")
print("  - Updated expected returns")
print("  - Risk tolerance changes")
print("\n" + "="*80)